# Library

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

import matplotlib.pyplot as plt
import seaborn as sns
import os
from copy import deepcopy
from tqdm import tqdm
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Recall

# Combination_Modeling

In [ ]:
# Define CNN Layersㅁ
def Cnn_layers(input_layer):
    conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    flatten = Flatten()(pool2)
    dropout = Dropout(0.5)(flatten)
    dense1 = Dense(128, activation='relu')(dropout)
    return dense1

# Define Dense MLP Layer
def Dense_layer(input_layer):
    dense0 = Dense(64, activation='relu')(input_layer)
    dense1 = Dense(64, activation='relu')(dense0)
    flatten = Flatten()(dense1)
    dense2 = Dense(64, activation='relu')(flatten)
    return dense2

# Load Training Data
path = f"Modeling_data/train/"
height_train = np.load(path + 'Height_train.npy')
ndvi_train = np.load(path + 'NDVI_train.npy')
slope_train = np.load(path + 'Slope_train.npy')
landuse_train = np.load(path + 'Landuse_train.npy')
popden_train = np.load(path + 'population_density_train.npy')

# Load and preprocess tabular training data
tabular_train = pd.read_csv(path + "climate_train.csv")
tabular_train.drop(['lon', 'lat'], axis=1, inplace=True)
tabular_train = tabular_train.replace(32767.0, -9999)

x_train, y_train = [], []
for j in tqdm(range(len(tabular_train))):
    x_train.append(np.array(tabular_train.loc[j, ['humidity', 'rainfall', 'temp', 'windspeed']]).astype(float))
    y_train.append(np.array(tabular_train.loc[j, ['target']]).astype(float))

climate_train = np.array(x_train)
y_train = np.array(y_train)

# Load Validation Data
val_path = "Modeling_data/val/"
height_val = np.load(val_path + 'Height_val.npy')
ndvi_val = np.load(val_path + 'NDVI_val.npy')
slope_val = np.load(val_path + 'Slope_val.npy')
landuse_val = np.load(val_path + 'Landuse_val.npy')
popden_val = np.load(val_path + 'population_density_val.npy')

# Load and preprocess tabular validation data
tabular_val = pd.read_csv(val_path + "climate_val.csv")
tabular_val.drop(['lon', 'lat'], axis=1, inplace=True)
tabular_val = tabular_val.replace(32767.0, -9999)

x_val, y_val = [], []
for j in tqdm(range(len(tabular_val))):
    x_val.append(np.array(tabular_val.loc[j, ['humidity', 'rainfall', 'temp', 'windspeed']]).astype(float))
    y_val.append(np.array(tabular_val.loc[j, ['target']]).astype(float))

climate_val = np.array(x_val)
y_val = np.array(y_val)

# Prepare training and validation data dictionaries
feature_dict_train = {
    "climate": climate_train,
    "height": height_train,
    "ndvi": ndvi_train,
    "slope": slope_train,
    "landuse": landuse_train,
    "popden": popden_train
}

# Prepare training and validation data dictionaries
feature_dict_val = {
    "climate": climate_val,
    "height": height_val,
    "ndvi": ndvi_val,
    "slope": slope_val,
    "landuse": landuse_val,
    "popden": popden_val
}

model_save_path = "Model_save/Combinations"
os.makedirs(model_save_path, exist_ok=True)

combinations = [
    ["climate"], ["height"],["ndvi"],["slope"],["landuse"],["popden"],["climate", "height"], ["climate", "ndvi"], ["climate", "slope"], ["climate", "landuse"], ["climate", "popden"],
    ["climate", "height", "ndvi"], ["climate", "height", "slope"], ["climate", "ndvi", "slope"], ["climate", "landuse", "popden"],
    ["climate", "height", "landuse"], ["climate", "height", "popden"], ["climate", "ndvi", "landuse"], ["climate", "ndvi", "popden"],
    ["climate", "slope", "landuse"], ["climate", "slope", "popden"], ["climate", "height", "ndvi", "landuse"],
    ["climate", "height", "ndvi", "popden"], ["climate", "height", "slope", "landuse"], ["climate", "height", "slope", "popden"],
    ["climate", "ndvi", "slope", "landuse"], ["climate", "ndvi", "slope", "popden"], ["climate","height","ndvi","slope","landuse"],["climate","height","ndvi","slope","popden"],
    ["climate","height","ndvi","slope"],["climate","height","ndvi","slope","landuse","popden"]
]

# Early Stopping and Learning Rate Reduction Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

for combo in combinations:
    train_data = {f"{feature}_input": feature_dict_train[feature] for feature in combo}
    val_data = {f"{feature}_input": feature_dict_val[feature] for feature in combo}  # 수정된 부분
    
    print(f"Training on combination: {combo}")
    print(f"train_data shapes: {[train_data[f'{feature}_input'].shape for feature in combo]}")
    print(f"val_data shapes: {[val_data[f'{feature}_input'].shape for feature in combo]}")

    inputs, processed_inputs = [], []
    for feature in combo:
        shape = (4, 1) if feature == "climate" else (24, 24, 3)
        input_layer = Input(shape=shape, name=f'{feature}_input')
        inputs.append(input_layer)
        
        if feature == "climate":
            processed_inputs.append(Dense_layer(input_layer))
        else:
            processed_inputs.append(Cnn_layers(input_layer))
    
    merged = concatenate(processed_inputs) if len(processed_inputs) > 1 else processed_inputs[0]
    output_layer = Dense(1, activation='sigmoid', name='output_layer')(merged)
    
    model = Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(train_data, y_train, validation_data=(val_data, y_val), epochs=50, batch_size=64, verbose=1, callbacks=[early_stopping, reduce_lr])
    
    model_name = "_".join(combo) + ".h5"
    model.save(os.path.join(model_save_path, model_name))

100%|██████████████████████████████████████████████████████████████████████████████| 422/422 [00:00<00:00, 1133.33it/s]


Training on combination: ['climate']
train_data shapes: [(1684, 4)]
val_data shapes: [(422, 4)]
Epoch 1/50
27/27 [==============================] - 1s 12ms/step - loss: 0.8102 - accuracy: 0.4400 - val_loss: 0.6984 - val_accuracy: 0.5024 - lr: 1.0000e-04
Epoch 2/50
27/27 [==============================] - 0s 3ms/step - loss: 0.6511 - accuracy: 0.6395 - val_loss: 0.6341 - val_accuracy: 0.6706 - lr: 1.0000e-04
Epoch 3/50
27/27 [==============================] - 0s 3ms/step - loss: 0.6051 - accuracy: 0.6894 - val_loss: 0.6220 - val_accuracy: 0.6659 - lr: 1.0000e-04
Epoch 4/50
27/27 [==============================] - 0s 4ms/step - loss: 0.5937 - accuracy: 0.6930 - val_loss: 0.6155 - val_accuracy: 0.6540 - lr: 1.0000e-04
Epoch 5/50
27/27 [==============================] - 0s 3ms/step - loss: 0.5879 - accuracy: 0.6865 - val_loss: 0.6177 - val_accuracy: 0.6280 - lr: 1.0000e-04
Epoch 6/50
27/27 [==============================] - 0s 4ms/step - loss: 0.5885 - accuracy: 0.6894 - val_loss: 0.6185 -

27/27 [==============================] - 24s 849ms/step - loss: 0.7037 - accuracy: 0.4863 - val_loss: 0.6959 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 2/50
27/27 [==============================] - 25s 924ms/step - loss: 0.7015 - accuracy: 0.4941 - val_loss: 0.6926 - val_accuracy: 0.5095 - lr: 1.0000e-04
Epoch 3/50
27/27 [==============================] - 24s 883ms/step - loss: 0.6978 - accuracy: 0.5024 - val_loss: 0.6929 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 4/50
27/27 [==============================] - 24s 895ms/step - loss: 0.6943 - accuracy: 0.5297 - val_loss: 0.6963 - val_accuracy: 0.4976 - lr: 1.0000e-04
Epoch 5/50
27/27 [==============================] - 24s 899ms/step - loss: 0.6999 - accuracy: 0.4875 - val_loss: 0.6923 - val_accuracy: 0.5142 - lr: 1.0000e-04
Epoch 6/50
27/27 [==============================] - 25s 902ms/step - loss: 0.6943 - accuracy: 0.5048 - val_loss: 0.6927 - val_accuracy: 0.5000 - lr: 1.0000e-04
Epoch 7/50
27/27 [==============================] -

27/27 [==============================] - 23s 845ms/step - loss: 0.6642 - accuracy: 0.6057 - val_loss: 0.6697 - val_accuracy: 0.5853 - lr: 1.0000e-04
Epoch 8/50
27/27 [==============================] - 22s 805ms/step - loss: 0.6644 - accuracy: 0.6015 - val_loss: 0.6552 - val_accuracy: 0.6185 - lr: 1.0000e-04
Epoch 9/50
27/27 [==============================] - 23s 869ms/step - loss: 0.6584 - accuracy: 0.6057 - val_loss: 0.6532 - val_accuracy: 0.6256 - lr: 1.0000e-04
Epoch 10/50
27/27 [==============================] - 23s 839ms/step - loss: 0.6570 - accuracy: 0.6146 - val_loss: 0.6515 - val_accuracy: 0.6232 - lr: 1.0000e-04
Epoch 11/50
27/27 [==============================] - 24s 884ms/step - loss: 0.6550 - accuracy: 0.5980 - val_loss: 0.6502 - val_accuracy: 0.6256 - lr: 1.0000e-04
Epoch 12/50
27/27 [==============================] - 22s 832ms/step - loss: 0.6519 - accuracy: 0.6051 - val_loss: 0.6499 - val_accuracy: 0.6280 - lr: 1.0000e-04
Epoch 13/50
27/27 [=============================

27/27 [==============================] - 25s 890ms/step - loss: 1.0216 - accuracy: 0.4846 - val_loss: 0.7175 - val_accuracy: 0.5308 - lr: 1.0000e-04
Epoch 2/50
27/27 [==============================] - 23s 858ms/step - loss: 0.6807 - accuracy: 0.5784 - val_loss: 0.6260 - val_accuracy: 0.6422 - lr: 1.0000e-04
Epoch 3/50
27/27 [==============================] - 22s 812ms/step - loss: 0.6010 - accuracy: 0.6918 - val_loss: 0.5914 - val_accuracy: 0.6754 - lr: 1.0000e-04
Epoch 4/50
27/27 [==============================] - 23s 839ms/step - loss: 0.5691 - accuracy: 0.7049 - val_loss: 0.5760 - val_accuracy: 0.6754 - lr: 1.0000e-04
Epoch 5/50
27/27 [==============================] - 24s 881ms/step - loss: 0.5554 - accuracy: 0.7055 - val_loss: 0.5654 - val_accuracy: 0.6991 - lr: 1.0000e-04
Epoch 6/50
27/27 [==============================] - 24s 880ms/step - loss: 0.5461 - accuracy: 0.7019 - val_loss: 0.5577 - val_accuracy: 0.6896 - lr: 1.0000e-04
Epoch 7/50
27/27 [==============================] -

...

27/27 [==============================] - 73s 3s/step - loss: 0.5193 - accuracy: 0.7518 - val_loss: 0.5337 - val_accuracy: 0.7275 - lr: 1.0000e-04
Epoch 7/50
27/27 [==============================] - 81s 3s/step - loss: 0.5183 - accuracy: 0.7405 - val_loss: 0.5318 - val_accuracy: 0.7251 - lr: 1.0000e-04
Epoch 8/50
27/27 [==============================] - 76s 3s/step - loss: 0.5153 - accuracy: 0.7429 - val_loss: 0.5329 - val_accuracy: 0.7204 - lr: 1.0000e-04
Epoch 9/50
27/27 [==============================] - 80s 3s/step - loss: 0.5174 - accuracy: 0.7411 - val_loss: 0.5288 - val_accuracy: 0.7346 - lr: 1.0000e-04
Epoch 10/50
27/27 [==============================] - 78s 3s/step - loss: 0.5120 - accuracy: 0.7470 - val_loss: 0.5350 - val_accuracy: 0.7204 - lr: 1.0000e-04
Epoch 11/50
27/27 [==============================] - 83s 3s/step - loss: 0.5101 - accuracy: 0.7482 - val_loss: 0.5261 - val_accuracy: 0.7393 - lr: 1.0000e-04
Epoch 12/50
27/27 [==============================] - 78s 3s/step - 

Epoch 18/50
27/27 [==============================] - 76s 3s/step - loss: 0.4965 - accuracy: 0.7583 - val_loss: 0.5144 - val_accuracy: 0.7393 - lr: 1.0000e-04
Epoch 19/50
27/27 [==============================] - 78s 3s/step - loss: 0.4984 - accuracy: 0.7571 - val_loss: 0.5167 - val_accuracy: 0.7346 - lr: 1.0000e-04
Epoch 20/50
27/27 [==============================] - 74s 3s/step - loss: 0.4933 - accuracy: 0.7613 - val_loss: 0.5130 - val_accuracy: 0.7370 - lr: 1.0000e-04
Epoch 21/50
27/27 [==============================] - 79s 3s/step - loss: 0.4953 - accuracy: 0.7482 - val_loss: 0.5134 - val_accuracy: 0.7417 - lr: 1.0000e-04
Epoch 22/50
27/27 [==============================] - 77s 3s/step - loss: 0.4938 - accuracy: 0.7542 - val_loss: 0.5125 - val_accuracy: 0.7393 - lr: 1.0000e-04
Epoch 23/50
27/27 [==============================] - 69s 3s/step - loss: 0.4905 - accuracy: 0.7607 - val_loss: 0.5115 - val_accuracy: 0.7417 - lr: 1.0000e-04
Epoch 24/50
27/27 [==============================] -

27/27 [==============================] - 61s 2s/step - loss: 0.5197 - accuracy: 0.7381 - val_loss: 0.5406 - val_accuracy: 0.7204 - lr: 1.0000e-04
Epoch 19/50
27/27 [==============================] - 64s 2s/step - loss: 0.5153 - accuracy: 0.7452 - val_loss: 0.5451 - val_accuracy: 0.7156 - lr: 1.0000e-04
Epoch 20/50
27/27 [==============================] - 70s 3s/step - loss: 0.5148 - accuracy: 0.7530 - val_loss: 0.5381 - val_accuracy: 0.7133 - lr: 1.0000e-04
Epoch 21/50
27/27 [==============================] - 61s 2s/step - loss: 0.5141 - accuracy: 0.7518 - val_loss: 0.5389 - val_accuracy: 0.7133 - lr: 1.0000e-04
Epoch 22/50
27/27 [==============================] - 64s 2s/step - loss: 0.5110 - accuracy: 0.7559 - val_loss: 0.5352 - val_accuracy: 0.7133 - lr: 1.0000e-04
Epoch 23/50
27/27 [==============================] - 62s 2s/step - loss: 0.5112 - accuracy: 0.7530 - val_loss: 0.5338 - val_accuracy: 0.7180 - lr: 1.0000e-04
Epoch 24/50
27/27 [==============================] - 65s 2s/step

27/27 [==============================] - 98s 4s/step - loss: 0.4919 - accuracy: 0.7625 - val_loss: 0.5226 - val_accuracy: 0.7370 - lr: 1.0000e-04
Epoch 19/50
27/27 [==============================] - 101s 4s/step - loss: 0.4955 - accuracy: 0.7666 - val_loss: 0.5169 - val_accuracy: 0.7441 - lr: 1.0000e-04
Epoch 20/50
27/27 [==============================] - 88s 3s/step - loss: 0.4916 - accuracy: 0.7648 - val_loss: 0.5143 - val_accuracy: 0.7488 - lr: 1.0000e-04
Epoch 21/50
27/27 [==============================] - 85s 3s/step - loss: 0.4890 - accuracy: 0.7607 - val_loss: 0.5218 - val_accuracy: 0.7299 - lr: 1.0000e-04
Epoch 22/50
27/27 [==============================] - 88s 3s/step - loss: 0.4908 - accuracy: 0.7714 - val_loss: 0.5183 - val_accuracy: 0.7441 - lr: 1.0000e-04
Epoch 23/50
27/27 [==============================] - 86s 3s/step - loss: 0.4925 - accuracy: 0.7577 - val_loss: 0.5155 - val_accuracy: 0.7441 - lr: 1.0000e-04
Epoch 24/50
27/27 [==============================] - 106s 4s/st

27/27 [==============================] - 96s 4s/step - loss: 0.5041 - accuracy: 0.7530 - val_loss: 0.5093 - val_accuracy: 0.7441 - lr: 1.0000e-04
Epoch 19/50
27/27 [==============================] - 96s 4s/step - loss: 0.4972 - accuracy: 0.7648 - val_loss: 0.5088 - val_accuracy: 0.7346 - lr: 1.0000e-04
Epoch 20/50
27/27 [==============================] - 92s 3s/step - loss: 0.4959 - accuracy: 0.7553 - val_loss: 0.5072 - val_accuracy: 0.7370 - lr: 1.0000e-04
Epoch 21/50
27/27 [==============================] - 98s 4s/step - loss: 0.5046 - accuracy: 0.7654 - val_loss: 0.5272 - val_accuracy: 0.7204 - lr: 1.0000e-04
Epoch 22/50
27/27 [==============================] - 91s 3s/step - loss: 0.5003 - accuracy: 0.7548 - val_loss: 0.5065 - val_accuracy: 0.7346 - lr: 1.0000e-04
Epoch 23/50
27/27 [==============================] - 95s 4s/step - loss: 0.4904 - accuracy: 0.7637 - val_loss: 0.5148 - val_accuracy: 0.7109 - lr: 1.0000e-04
Epoch 24/50
27/27 [==============================] - 99s 4s/step

# Threshold_Test

In [6]:
combinations = [
    ["climate"], ["height"],["ndvi"],["slope"],["landuse"],["popden"],["climate", "height"], ["climate", "ndvi"], ["climate", "slope"], ["climate", "landuse"], ["climate", "popden"],
    ["climate", "height", "ndvi"], ["climate", "height", "slope"], ["climate", "ndvi", "slope"], ["climate", "landuse", "popden"],
    ["climate", "height", "landuse"], ["climate", "height", "popden"], ["climate", "ndvi", "landuse"], ["climate", "ndvi", "popden"],
    ["climate", "slope", "landuse"], ["climate", "slope", "popden"], ["climate", "height", "ndvi", "landuse"],
    ["climate", "height", "ndvi", "popden"], ["climate", "height", "slope", "landuse"], ["climate", "height", "slope", "popden"],
    ["climate", "ndvi", "slope", "landuse"], ["climate", "ndvi", "slope", "popden"], ["climate","height","ndvi","slope","landuse"],["climate","height","ndvi","slope","popden"],
    ["climate","height","ndvi","slope"],["climate","height","ndvi","slope","landuse","popden"]
]

model_load_path = "Model_save/Combinations"

# Model Evaluation
thresholds = [0.3, 0.4, 0.45, 0.5, 0.55, 0.6]
results = []

for combo in combinations:
    model_name = "_".join(combo) + ".h5"
    model = load_model(os.path.join(model_load_path, model_name))
    
    # Ensure val_data only includes features from the trained combination
    val_data = {f"{feature}_input": feature_dict_val[feature] for feature in combo}  
    
    print(f"Evaluating model trained on: {combo}")
    print(f"Validation data keys: {list(val_data.keys())}")
    
    val_predictions = model.predict(val_data)
    
    for threshold in thresholds:
        val_predictions_binary = (val_predictions > threshold).astype(int)
        report = classification_report(y_val, val_predictions_binary, output_dict=True)
        
        accuracy = accuracy_score(y_val, val_predictions_binary)
        precision_0 = report["0.0"]["precision"]
        recall_0 = report["0.0"]["recall"]
        f1_0 = report["0.0"]["f1-score"]
        precision_1 = report["1.0"]["precision"]
        recall_1 = report["1.0"]["recall"]
        f1_1 = report["1.0"]["f1-score"]
        
        macro_precision = report["macro avg"]["precision"]
        macro_recall = report["macro avg"]["recall"]
        macro_f1 = report["macro avg"]["f1-score"]
        
        weighted_precision = report["weighted avg"]["precision"]
        weighted_recall = report["weighted avg"]["recall"]
        weighted_f1 = report["weighted avg"]["f1-score"]
        
        results.append([combo, threshold, accuracy, precision_0, recall_0, f1_0, precision_1, recall_1, f1_1,
                        macro_precision, macro_recall, macro_f1, weighted_precision, weighted_recall, weighted_f1])

        print(f"Combination {combo}, Threshold {threshold}: Accuracy: {accuracy}")
        print(f"Class 0 - Precision: {precision_0}, Recall: {recall_0}, F1 Score: {f1_0}")
        print(f"Class 1 - Precision: {precision_1}, Recall: {recall_1}, F1 Score: {f1_1}")
        print(f"Macro Avg - Precision: {macro_precision}, Recall: {macro_recall}, F1 Score: {macro_f1}")
        print(f"Weighted Avg - Precision: {weighted_precision}, Recall: {weighted_recall}, F1 Score: {weighted_f1}")

# Save Evaluation Results
results_df = pd.DataFrame(results, columns=["Feature Combination", "Threshold", "Accuracy", "Precision_0", "Recall_0", "F1_0", "Precision_1", "Recall_1", "F1_1",
                                            "Macro Precision", "Macro Recall", "Macro F1", "Weighted Precision", "Weighted Recall", "Weighted F1"])
results_df.to_csv("Model_result/threshold_evaluation_results.csv", index=False)
print("Results saved to model_evaluation_results.csv")


100%|██████████████████████████████████████████████████████████████████████████████| 422/422 [00:00<00:00, 2785.50it/s]


Evaluating model trained on: ['climate']
Validation data keys: ['climate_input']
14/14 [==============================] - 0s 935us/step
Combination ['climate'], Threshold 0.3: Accuracy: 0.6398104265402843
Class 0 - Precision: 0.736, Recall: 0.43601895734597157, F1 Score: 0.5476190476190476
Class 1 - Precision: 0.5993265993265994, Recall: 0.8436018957345972, F1 Score: 0.7007874015748031
Macro Avg - Precision: 0.6676632996632996, Recall: 0.6398104265402844, F1 Score: 0.6242032245969253
Weighted Avg - Precision: 0.6676632996632996, Recall: 0.6398104265402843, F1 Score: 0.6242032245969255
Combination ['climate'], Threshold 0.4: Accuracy: 0.6611374407582938
Class 0 - Precision: 0.6717171717171717, Recall: 0.6303317535545023, F1 Score: 0.6503667481662592
Class 1 - Precision: 0.6517857142857143, Recall: 0.6919431279620853, F1 Score: 0.671264367816092
Macro Avg - Precision: 0.6617514430014431, Recall: 0.6611374407582938, F1 Score: 0.6608155579911756
Weighted Avg - Precision: 0.6617514430014431

14/14 [==============================] - 0s 2ms/step


C:\Users\EBDL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\EBDL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\EBDL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Combination ['slope'], Threshold 0.3: Accuracy: 0.6279620853080569
Class 0 - Precision: 0.7647058823529411, Recall: 0.3696682464454976, F1 Score: 0.4984025559105431
Class 1 - Precision: 0.584375, Recall: 0.8862559241706162, F1 Score: 0.704331450094162
Macro Avg - Precision: 0.6745404411764706, Recall: 0.6279620853080569, F1 Score: 0.6013670030023526
Weighted Avg - Precision: 0.6745404411764706, Recall: 0.6279620853080569, F1 Score: 0.6013670030023525
Combination ['slope'], Threshold 0.4: Accuracy: 0.6800947867298578
Class 0 - Precision: 0.7345679012345679, Recall: 0.5639810426540285, F1 Score: 0.6380697050938338
Class 1 - Precision: 0.6461538461538462, Recall: 0.7962085308056872, F1 Score: 0.713375796178344
Macro Avg - Precision: 0.6903608736942071, Recall: 0.6800947867298579, F1 Score: 0.6757227506360889
Weighted Avg - Precision: 0.6903608736942071, Recall: 0.6800947867298578, F1 Score: 0.675722750636089
Combination ['slope'], Threshold 0.45: Accuracy: 0.6990521327014217
Class 0 - Pre

...

Evaluating model trained on: ['climate', 'ndvi', 'popden']
Validation data keys: ['climate_input', 'ndvi_input', 'popden_input']
14/14 [==============================] - 0s 2ms/step
Combination ['climate', 'ndvi', 'popden'], Threshold 0.3: Accuracy: 0.6184834123222749
Class 0 - Precision: 0.7604166666666666, Recall: 0.3459715639810427, F1 Score: 0.4755700325732899
Class 1 - Precision: 0.5766871165644172, Recall: 0.8909952606635071, F1 Score: 0.7001862197392922
Macro Avg - Precision: 0.668551891615542, Recall: 0.6184834123222749, F1 Score: 0.5878781261562911
Weighted Avg - Precision: 0.6685518916155418, Recall: 0.6184834123222749, F1 Score: 0.5878781261562911
Combination ['climate', 'ndvi', 'popden'], Threshold 0.4: Accuracy: 0.6611374407582938
Class 0 - Precision: 0.717948717948718, Recall: 0.5308056872037915, F1 Score: 0.6103542234332425
Class 1 - Precision: 0.6278195488721805, Recall: 0.7914691943127962, F1 Score: 0.70020964360587
Macro Avg - Precision: 0.6728841334104492, Recall: 0.

Evaluating model trained on: ['climate', 'height', 'ndvi', 'landuse']
Validation data keys: ['climate_input', 'height_input', 'ndvi_input', 'landuse_input']
14/14 [==============================] - 0s 3ms/step
Combination ['climate', 'height', 'ndvi', 'landuse'], Threshold 0.3: Accuracy: 0.7014218009478673
Class 0 - Precision: 0.8244274809160306, Recall: 0.5118483412322274, F1 Score: 0.631578947368421
Class 1 - Precision: 0.6460481099656358, Recall: 0.8909952606635071, F1 Score: 0.749003984063745
Macro Avg - Precision: 0.7352377954408331, Recall: 0.7014218009478672, F1 Score: 0.690291465716083
Weighted Avg - Precision: 0.7352377954408332, Recall: 0.7014218009478673, F1 Score: 0.690291465716083
Combination ['climate', 'height', 'ndvi', 'landuse'], Threshold 0.4: Accuracy: 0.7464454976303317
Class 0 - Precision: 0.8023255813953488, Recall: 0.6540284360189573, F1 Score: 0.7206266318537858
Class 1 - Precision: 0.708, Recall: 0.8388625592417062, F1 Score: 0.7678958785249457
Macro Avg - Prec

Evaluating model trained on: ['climate', 'height', 'slope', 'popden']
Validation data keys: ['climate_input', 'height_input', 'slope_input', 'popden_input']
14/14 [==============================] - 0s 8ms/step
Combination ['climate', 'height', 'slope', 'popden'], Threshold 0.3: Accuracy: 0.7322274881516587
Class 0 - Precision: 0.8769230769230769, Recall: 0.5402843601895735, F1 Score: 0.6686217008797655
Class 1 - Precision: 0.6678082191780822, Recall: 0.9241706161137441, F1 Score: 0.7753479125248508
Macro Avg - Precision: 0.7723656480505796, Recall: 0.7322274881516588, F1 Score: 0.7219848067023081
Weighted Avg - Precision: 0.7723656480505795, Recall: 0.7322274881516587, F1 Score: 0.7219848067023082
Combination ['climate', 'height', 'slope', 'popden'], Threshold 0.4: Accuracy: 0.7630331753554502
Class 0 - Precision: 0.8284023668639053, Recall: 0.6635071090047393, F1 Score: 0.7368421052631579
Class 1 - Precision: 0.7193675889328063, Recall: 0.8625592417061612, F1 Score: 0.7844827586206896

Evaluating model trained on: ['climate', 'height', 'ndvi', 'slope', 'landuse']
Validation data keys: ['climate_input', 'height_input', 'ndvi_input', 'slope_input', 'landuse_input']
14/14 [==============================] - 0s 3ms/step
Combination ['climate', 'height', 'ndvi', 'slope', 'landuse'], Threshold 0.3: Accuracy: 0.7417061611374408
Class 0 - Precision: 0.8541666666666666, Recall: 0.5829383886255924, F1 Score: 0.6929577464788732
Class 1 - Precision: 0.6834532374100719, Recall: 0.9004739336492891, F1 Score: 0.7770961145194274
Macro Avg - Precision: 0.7688099520383693, Recall: 0.7417061611374407, F1 Score: 0.7350269304991504
Weighted Avg - Precision: 0.7688099520383693, Recall: 0.7417061611374408, F1 Score: 0.7350269304991502
Combination ['climate', 'height', 'ndvi', 'slope', 'landuse'], Threshold 0.4: Accuracy: 0.7677725118483413
Class 0 - Precision: 0.8192090395480226, Recall: 0.6872037914691943, F1 Score: 0.7474226804123711
Class 1 - Precision: 0.7306122448979592, Recall: 0.8483

Evaluating model trained on: ['climate', 'height', 'ndvi', 'slope', 'landuse', 'popden']
Validation data keys: ['climate_input', 'height_input', 'ndvi_input', 'slope_input', 'landuse_input', 'popden_input']
14/14 [==============================] - 0s 9ms/step
Combination ['climate', 'height', 'ndvi', 'slope', 'landuse', 'popden'], Threshold 0.3: Accuracy: 0.7203791469194313
Class 0 - Precision: 0.872, Recall: 0.5165876777251185, F1 Score: 0.6488095238095238
Class 1 - Precision: 0.6565656565656566, Recall: 0.9241706161137441, F1 Score: 0.7677165354330707
Macro Avg - Precision: 0.7642828282828282, Recall: 0.7203791469194313, F1 Score: 0.7082630296212973
Weighted Avg - Precision: 0.7642828282828283, Recall: 0.7203791469194313, F1 Score: 0.7082630296212973
Combination ['climate', 'height', 'ndvi', 'slope', 'landuse', 'popden'], Threshold 0.4: Accuracy: 0.7440758293838863
Class 0 - Precision: 0.8198757763975155, Recall: 0.6255924170616114, F1 Score: 0.7096774193548386
Class 1 - Precision: 0

# PDP_Analysis

In [7]:
# ---------- 1. Metric ----------

def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'))
    precision = tp / (K.sum(K.cast(y_pred, 'float')) + K.epsilon())
    recall    = tp / (K.sum(K.cast(y_true, 'float')) + K.epsilon())
    return 2 * (precision * recall) / (precision + recall + K.epsilon())

# ---------- 2. Model ----------
model_path = 'Model_save/WLH_Model.h5'
model = load_model(model_path, custom_objects={'f1_score': f1_score})
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy', Recall(name='recall'), f1_score])

# ---------- 3. Data ----------
val_path = 'Modeling_data/val/'
height_val = np.load(os.path.join(val_path, 'Height_val.npy'))
ndvi_val   = np.load(os.path.join(val_path, 'NDVI_val.npy'))
slope_val  = np.load(os.path.join(val_path, 'Slope_val.npy'))
landuse_val= np.load(os.path.join(val_path, 'Landuse_val.npy'))
popden_val = np.load(os.path.join(val_path, 'population_density_val.npy'))

tabular_val = pd.read_csv(os.path.join(val_path, 'climate_val.csv'))
tabular_val.drop(['lon', 'lat'], axis=1, inplace=True)
tabular_val = tabular_val.replace(32767.0, -9999)
climate_val = tabular_val[['humidity', 'rainfall', 'temp', 'windspeed']].values.astype(float)
climate_val = climate_val.reshape(-1, 4, 1)

val = {
    'height_input': height_val,
    'ndvi_input'  : ndvi_val,
    'slope_input' : slope_val,
    'landuse_input': landuse_val,
    'popden_input': popden_val,
    'climate_input': climate_val,
}

# ---------- 4. PDP helpers ----------
img_keys = ['height_input','ndvi_input','slope_input','popden_input']

def pdp_img_channel_mean_norm(model, val_dict, f_key, n_points=30):
    ch_mean = val_dict[f_key].mean(axis=(1,2))
    grid = np.linspace(0, 1, n_points)
    scores=[]
    for g in np.nditer(grid):
        delta = (g - ch_mean)[:,None,None,:]
        X_tmp = deepcopy(val_dict)
        X_tmp[f_key] = X_tmp[f_key] + delta
        scores.append(model.predict(X_tmp, verbose=0).mean())
    return grid.tolist(), scores

climate_idx = {'humidity':1,'rainfall':0,'temp':2,'windspeed':3}

def pdp_climate(model,val_dict,var,n_points=30):
    idx=climate_idx[var]
    feat=val_dict['climate_input'][:,idx,0]
    grid=np.linspace(feat.min(),feat.max(),n_points)
    scores=[]
    for g in np.nditer(grid):
        X_tmp=deepcopy(val_dict)
        X_tmp['climate_input'][:,idx,0]=g
        scores.append(model.predict(X_tmp,verbose=0).mean())
    return grid.tolist(), scores

# Land‑use class substitution scores
class_palette = {
    'Human'      : np.array([255,   0,   0])/255.0,
    'Industrial' : np.array([255, 255,   0])/255.0,
    'Farmland'   : np.array([139, 115,  85])/255.0,
    'Greenery'   : np.array([  0, 128,   0])/255.0,
    'Etc'        : np.array([  0,   0, 255])/255.0
}

def landuse_class_scores(model,val_dict):
    scores={}
    for cls,rgb in class_palette.items():
        X_tmp=deepcopy(val_dict)
        X_tmp['landuse_input'][:]=rgb
        scores[cls]=model.predict(X_tmp,verbose=0).mean()
    return scores

# ---------- 5. Feature plotting order (9 total) ----------
features = [
    ('temp','Temperature'),
    ('humidity','Humidity'),
    ('windspeed','Windspeed'),
    ('rainfall','Precipitation'),
    ('height_input','Elevation'),
    ('ndvi_input','NDVI'),
    ('slope_input','Slope'),
    ('popden_input','Population density'),
    ('landuse','Land use')
]

# ---------- 6. Plot ----------
plt.rcParams.update({'figure.dpi':300,'axes.titlesize':10,'axes.labelsize':9,
                     'xtick.labelsize':8,'ytick.labelsize':8,'axes.linewidth':0.6})
sns.set_style('whitegrid',{'axes.grid':False})
fig,axes=plt.subplots(3,3,figsize=(6.5,6.5)); axes=axes.flatten()
clr=sns.color_palette('Blues',5)[3]

for i,(key,label) in enumerate(features):
    ax=axes[i]
    if key in img_keys:
        grid,sc=pdp_img_channel_mean_norm(model,val,key)
        ax.plot(grid,sc,color=clr,linewidth=2)
        ax.set_xticks([0,0.5,1])
    elif key in climate_idx:
        grid,sc=pdp_climate(model,val,key)
        mid=grid[len(grid)//2]
        ax.plot(grid,sc,color=clr,linewidth=2)
        ax.set_xticks([grid[0],mid,grid[-1]])
    else:  # landuse bar
        scores=landuse_class_scores(model,val)
        ax.bar(list(scores.keys()), list(scores.values()), color=clr)
        ax.set_xticks(range(len(scores)))
        ax.set_xticklabels(list(scores.keys()), rotation=45, ha='right')
    ax.set_ylim(0,1)
    ax.set_title(label,pad=2,fontsize=9)
    ax.tick_params(length=2)

plt.subplots_adjust(wspace=0.25,hspace=0.4,left=0.1,right=0.97,top=0.95,bottom=0.15)
plt.savefig('Figure_PDP_9vars.png',dpi=300)
plt.close()

print('✅ Figure_PDP_9vars.png Save')


✅ Figure_PDP_9vars.png Save
